In [1]:
from solid import (
    circle,
    cylinder,
    translate,
    cube,
    rotate,
    rotate_extrude,
    square,
    polygon,
    sphere, 
    intersection
)
import viewscad
r = viewscad.Renderer()

In [2]:
# define parameters and tube function
height = 10
thick = 3
dowel_od = 13.5
dowel_od_o = dowel_od + 2*thick
screw_od = 3.5
arm_len = 20

def tube(od, id, height): return cylinder(d=od, h=height) - cylinder(d=id, h=height)

In [12]:


# Main dowel snap on arc ---------------------------------------------------
support_cylinder = tube(dowel_od_o, dowel_od, height)

# screw to hold support piece in place
screw = cylinder(d=screw_od, h=50)
screw = rotate([90,0,0])(screw)
screw = translate([0,0,height/2])(screw)
part = support_cylinder - screw

# removing arc for snap on
x_sect = rotate_extrude(angle = 90)(square(18))
x_sect = rotate([0,0,45])(x_sect)
part -= x_sect

# support for cord -------------------------------------------------------------------------
cord_cube = cube([arm_len, thick, height])
cord_cube = translate([dowel_od/2,-dowel_od/3,0])(cord_cube)
cord_cube += translate([0,dowel_od/2,0])(cord_cube)
part += cord_cube


# command strip base -------------------------------------------------------------------------
command_height, command_width = 50, 20
command_base = cube([thick, command_width, command_height], center=True)
command_base -= translate([thick,0,0])(rotate([0,-90,0])(cylinder(d=screw_od, h=50)))   # screw hole 1
command_base -= translate([thick,0,command_height/4])(rotate([0,-90,0])(cylinder(d=screw_od, h=50)))   # screw hole 2


command_base = translate([dowel_od/2+arm_len,0,command_height/2])(command_base)

part += command_base

stl_file = r.render(
    part,
    file_header="$fa=.01;\n $fs=0.01",
    outfile=f"CommandDowel.stl",
)